In [2]:
import os
import pandas as pd

# Folder containing all raw patient files
raw_folder = r'C:\PYTHON HACK'
cleaned_folder = r'C:\PYTHON HACK\Cleaned_data'
os.makedirs(cleaned_folder, exist_ok=True)

# Loop through all CSVs
for file in os.listdir(raw_folder):
    if file.lower().endswith('.csv'):
        file_path = os.path.join(raw_folder, file)
        print(f"🧼 Cleaning {file}")

        try:
            # Load with flexible delimiter handling
            df = pd.read_csv(file_path, sep=None, engine='python', on_bad_lines='skip')

            # Standardize column names
            df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

            # Parse time column if present
            time_cols = [col for col in df.columns if 'time' in col]
            if time_cols:
                df[time_cols[0]] = pd.to_datetime(df[time_cols[0]], errors='coerce')
                df.dropna(subset=[time_cols[0]], inplace=True)

            # Drop fully empty rows
            df.dropna(how='all', inplace=True)

            # Save cleaned file
            cleaned_path = os.path.join(cleaned_folder, f'cleaned_{file}')
            df.to_csv(cleaned_path, index=False)
            print(f"✅ Saved: {cleaned_path}")

        except Exception as e:
            print(f"❌ Failed to clean {file}: {e}")


🧼 Cleaning HUPA0001P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0001P.csv
🧼 Cleaning HUPA0002P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0002P.csv
🧼 Cleaning HUPA0003P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0003P.csv
🧼 Cleaning HUPA0004P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0004P.csv
🧼 Cleaning HUPA0005P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0005P.csv
🧼 Cleaning HUPA0006P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0006P.csv
🧼 Cleaning HUPA0007P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0007P.csv
🧼 Cleaning HUPA0009P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0009P.csv
🧼 Cleaning HUPA0010P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0010P.csv
🧼 Cleaning HUPA0011P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0011P.csv
🧼 Cleaning HUPA0014P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA0014P.csv
🧼 Cleaning HUPA0015P.csv
✅ Saved: C:\PYTHON HACK\Cleaned_data\cleaned_HUPA00

In [3]:
import os
import pandas as pd
import glob

# Path to cleaned files
cleaned_folder = r'C:\PYTHON HACK\Cleaned_data'

# Grab all patient files (excluding demographics)
patient_files = glob.glob(os.path.join(cleaned_folder, 'cleaned_HUPA*.csv'))

# Merge all into one DataFrame
merged_df = pd.concat([pd.read_csv(f) for f in patient_files], ignore_index=True)

# Optional: Add filename as patient ID if not already present
merged_df['source_file'] = [os.path.basename(f) for f in patient_files for _ in range(len(pd.read_csv(f)))]

# Save merged file
merged_path = os.path.join(cleaned_folder, 'merged_patients.csv')
merged_df.to_csv(merged_path, index=False)

print(f"✅ Merged file saved at: {merged_path}")


✅ Merged file saved at: C:\PYTHON HACK\Cleaned_data\merged_patients.csv


In [4]:
merged_df['patient_id'] = merged_df['source_file'].str.extract(r'(HUPA\d{4}P)')


In [6]:
import glob
import pandas as pd

all_files = glob.glob(r'C:\PYTHON HACK\Cleaned_data\cleaned_HUPA*.csv')
merged_df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
merged_df.to_csv(r'C:\PYTHON HACK\Cleaned_data\merged_patients.csv', index=False)


In [9]:
import os

# Add source file name to each row
merged_df['source_file'] = [os.path.basename(f) for f in all_files for _ in range(len(pd.read_csv(f)))]

# Extract patient_id from filename (e.g., HUPA0001P)
merged_df['patient_id'] = merged_df['source_file'].str.extract(r'(HUPA\d{4}P)')


In [11]:
print("Matched rows:", final_df['age'].notna().sum())
print("Unmatched rows:", final_df['age'].isna().sum())


Matched rows: 309392
Unmatched rows: 0


In [5]:
import pandas as pd

# Load your dataset
df = pd.read_csv(r'C:\PYTHON HACK\Cleaned_data\final_dataset.csv')

# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Create new columns
df['date'] = df['time'].dt.date
df['clock_time'] = df['time'].dt.time
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.day_name()

# Save the updated dataset to Excel
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_time_features.xlsx', index=False)

print("✅ Time features added and saved to Excel!")


✅ Time features added and saved to Excel!


In [6]:
import pandas as pd

# Load your dataset
df = pd.read_csv(r'C:\PYTHON HACK\Cleaned_data\final_dataset.csv')

# Step 1: Fill missing glucose values with the median
df['glucose'] = df['glucose'].fillna(df['glucose'].median())

# Step 2: Flag glucose levels
df['glucose_flag'] = pd.cut(
    df['glucose'],
    bins=[0, 70, 180, 300],
    labels=['Low', 'Normal', 'High'],
    include_lowest=True
)

# Step 3: Convert mg/dL to mmol/L (1 mmol/L = 18 mg/dL)
df['glucose_mmol'] = df['glucose'] / 18

# Optional: Round mmol values for cleaner display
df['glucose_mmol'] = df['glucose_mmol'].round(2)

# Save updated file
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_glucose.xlsx', index=False)

print("✅ Glucose features added and saved!")


✅ Glucose features added and saved!


In [7]:
# Normalize calories to a 0–1 scale
df['calories_norm'] = (df['calories'] - df['calories'].min()) / (df['calories'].max() - df['calories'].min())

# Flag days with high calorie burn
df['high_burn'] = df['calories'] > 2500


In [8]:
# Apply rolling average with a window of 5 readings
df['hr_rolling'] = df['heart_rate'].rolling(window=5, min_periods=1).mean()


In [9]:
import pandas as pd

# Load your dataset
df = pd.read_csv(r'C:\PYTHON HACK\Cleaned_data\final_dataset.csv')

# Step 1: Convert 'time' column to datetime if not already
df['time'] = pd.to_datetime(df['time'], errors='coerce')

# Step 2: Sort by time to ensure rolling window follows chronological order
df = df.sort_values('time')

# Step 3: Apply rolling average with a window of 5 readings
df['hr_rolling'] = df['heart_rate'].rolling(window=5, min_periods=1).mean()

# Optional: Round for cleaner display
df['hr_rolling'] = df['hr_rolling'].round(2)

# Save the updated file
df.to_excel(r'C:\PYTHON HACK\Cleaned_data\final_dataset_with_hr_smooth.xlsx', index=False)

print("✅ Heart rate smoothed and saved!")


✅ Heart rate smoothed and saved!
